In [ ]:
import joblib
import json
import pandas as pd
import pickle

### Table of contents
[1. Deserialize the model stuff](#1.-Deserialize-the-model-stuff)   
[2. Deserialize and prepare the observation](#2.-Deserialize-and-prepare-the-observation)

## 1. Deserialize the model stuff

Now here's a pretty cool thing: we are going to predict on a new
observation with only the imports we have above. Notice that there's
nothing about scikit up there.

If you can remember from the previous notebook, there were 3
things that we serialized, so now we need to deserialize them.

Firstly, let's deserialize the columns from the super handy json format:

In [ ]:
with open('columns.json', 'r') as fh:
    columns = json.load(fh)
columns

And now let's un-pickle the dtypes as well:

In [ ]:
with open('dtypes.pickle', 'rb') as fh:
    dtypes = pickle.load(fh)
dtypes    

Finally let's load the pipeline:

In [ ]:
pipeline = joblib.load('pipeline.pickle')
pipeline

Okay, now we've got everything that we need in order to make
a new prediction! All that's needed is the prediction itself,
which should come in a json format.

## 2. Deserialize and prepare the observation

This is exactly as we saw in the previous notebook:

In [ ]:
new_obs_str = '{"Age": 22.0, "Cabin": null, "Embarked": "S", "Fare": 7.25, "Parch": 0, "Pclass": 3, "Sex": "male", "SibSp": 1}'
new_obs_dict = json.loads(new_obs_str)
obs = pd.DataFrame([new_obs_dict], columns=columns)
obs = obs.astype(dtypes)

Which leaves us with a dataframe containing a single observation:

In [ ]:
obs

Which in turn can be passed into predict_proba:

In [ ]:
outcome = pipeline.predict_proba(obs)
outcome

In a binary classification task, the outcome array has two entries for each observation: the 0th entry corresponds to the probability of being in the *negative class (or 0 in our case*), while the 1st corresponds to the probability of being in the *positive class (or 1 in our case)*.

So, if we want to know the likelihood of surviving, we need to look at the probability of being in the positive class, which you can access like this:

In [ ]:
# there's only a single observation... so yeah
observation_index = 0
# This is the trick, go for the the positive class index
positive_class_index = 1
# Indexing in numpy arrays is a bit different than in normal
# python arrays:
survival_probability = outcome[observation_index, positive_class_index]
print('The observation has {} probability of survival.'.format(survival_probability))